In [23]:
import numpy as np
import os
import cv2

In [24]:
a = os.listdir(".")
def csv_gen(path):
    return ( i for i in np.genfromtxt(path, delimiter=','))

def fname_iter(directory):
    return (d for d in os.listdir(directory))
def img_iter(directory):
    im_exts = ['png','jpg','jpeg' ]
    for path in fname_iter(directory):
        if path.split('.')[-1] in im_exts:
            im = cv2.imread(path,mode='RGB')
            yield im

data_path = [
    './data/ML_interfere_eval/seq/data_20-03-2019_18-59_portD1.csv'
    ,'./data/ML_interfere_eval/seq/data_20-03-2019_18-59_portD2.csv'
]

In [30]:
def read_csv_data(filepath):
    d = np.genfromtxt(filepath,delimiter=',')
    return d

data = [
    read_csv_data(p) for p in data_path
]

In [29]:
data

[array([[1.70314700e+01, 1.55309610e+09],
        [1.68253500e+01, 1.55309610e+09],
        [1.70440400e+01, 1.55309610e+09],
        ...,
        [1.34020100e+01, 1.55309753e+09],
        [1.33090200e+01, 1.55309753e+09],
        [1.33982900e+01, 1.55309753e+09]]),
 array([[4.08076000e+00, 1.55309609e+09],
        [3.97854000e+00, 1.55309609e+09],
        [3.66311000e+00, 1.55309610e+09],
        ...,
        [6.16109000e+00, 1.55309753e+09],
        [5.90483000e+00, 1.55309753e+09],
        [5.66042000e+00, 1.55309753e+09]])]